In [31]:
!pip install --quiet geopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from geopy import distance

C = [
        [0, 0.2, 0.4, 0.6, 0.3, 0.2, 0.5, 0.4, 0.6, 0.3],
        [0.2, 0, 0.2, 0.4, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],
        [0.4, 0.2, 0, 0.2, 0.2, 0.4, 0.3, 0.5, 0.5, 0.6],
        [0.6, 0.4, 0.2, 0, 0.3, 0.4, 0.2, 0.4, 0.3, 0.6],
        [0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.3, 0.4, 0.4],
        [0.2, 0.3, 0.4, 0.4, 0.2, 0, 0.3, 0.2, 0.5, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.5],
        [0.4, 0.5, 0.5, 0.4, 0.3, 0.2, 0.2, 0, 0.3, 0.3],
        [0.6, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.3, 0, 0.6],
        [0.3, 0.5, 0.6, 0.6, 0.4, 0.2, 0.5, 0.3, 0.6, 0]
    ]

n=10
shape = np.shape(C)
X = cp.Variable(shape, boolean=True)
u = cp.Variable(n, nonneg=True)
d = cp.Variable(n, nonneg=True)  # duration time
w = cp.Variable(n, nonneg=True)  # waiting time
ones = np.ones((n,1))

# Defining the objective function
# multiplying 20 to calculate the time from distance according to the walking speed
# walking time is 3 mile per hour, which also means 1 mile per "20" minutes
objective = 20*cp.sum(cp.multiply(C, X)) + w[0]+w[1]+w[2]+w[3]+w[4]+w[5]+w[6]+w[7]+w[8]+w[9] + d[0]+d[1]+d[2]+d[3]+d[4]+d[5]+d[6]+d[7]+d[8]+d[9]

# Defining the constraints
constraints = []
constraints += [X @ ones == ones]
constraints += [X.T @ ones == ones]
constraints += [cp.diag(X) == 0]
constraints += [u[1:] >= 2]
constraints += [u[1:] <= n]
constraints += [u[0] == 1]

constraints += [d[0] >= 2]
constraints += [d[1] >= 4]
constraints += [d[2] >= 2]
constraints += [d[3] >= 3]
constraints += [d[4] >= 4]
constraints += [d[5] >= 5]
constraints += [d[6] >= 6]
constraints += [d[7] >= 2]
constraints += [d[8] >= 2]
constraints += [d[9] >= 3]

constraints += [w[0] >= 20]
constraints += [w[1] >= 10]
constraints += [w[2] >= 15]
constraints += [w[3] >= 10]
constraints += [w[4] >= 10]
constraints += [w[5] >= 5]
constraints += [w[6] >= 10]
constraints += [w[7] >= 20]
constraints += [w[8] >= 15]
constraints += [w[9] >= 15]

for i in range(1, n):
    for j in range(1, n):
        if i != j:
            constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

# Solving the problem
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver=cp.GUROBI,verbose = False)

# Transforming the solution to a path
X_sol = np.argwhere(X.value==1)
order = X_sol[0].tolist()

for i in range(1, n):
    row = order[-1]
    order.append(X_sol[row,1])

# Showing the optimal path
print('The path is:\n')
print( ' => '.join(map(str, order)))

distance = np.sum(np.multiply(C, X.value))
print('The optimal distance is:', np.round(distance,2), 'mile')

print("objective value: ", objective.value, "minutes")
print("optimal duration per each ride: ", d.value)
print("optimal waiting per each ride: ", w.value)

The path is:

0 => 1 => 4 => 2 => 3 => 6 => 8 => 7 => 9 => 5 => 0
The optimal distance is: 2.2 mile
objective value:  207.0 minutes
optimal duration per each ride:  [2. 4. 2. 3. 4. 5. 6. 2. 2. 3.]
optimal waiting per each ride:  [20. 10. 15. 10. 10.  5. 10. 20. 15. 15.]


In [32]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming

distance = np.array([
        [0, 0.2, 0.4, 0.6, 0.3, 0.2, 0.5, 0.4, 0.6, 0.3],
        [0.2, 0, 0.2, 0.4, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],
        [0.4, 0.2, 0, 0.2, 0.2, 0.4, 0.3, 0.5, 0.5, 0.6],
        [0.6, 0.4, 0.2, 0, 0.3, 0.4, 0.2, 0.4, 0.3, 0.6],
        [0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.3, 0.4, 0.4],
        [0.2, 0.3, 0.4, 0.4, 0.2, 0, 0.3, 0.2, 0.5, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.5],
        [0.4, 0.5, 0.5, 0.4, 0.3, 0.2, 0.2, 0, 0.3, 0.3],
        [0.6, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.3, 0, 0.6],
        [0.3, 0.5, 0.6, 0.6, 0.4, 0.2, 0.5, 0.3, 0.6, 0]
    ])

permutation, distance = solve_tsp_dynamic_programming(distance)
print("optimal path: ", permutation)
print("total distance: ", distance, "mile")

optimal path:  [0, 1, 4, 2, 3, 6, 8, 5, 7, 9]
total distance:  2.1999999999999997 mile


The tsp package travels around every node once and go back to the starting point at the end. The permutation of the tsp doesn't print out the last path, which is a path between the last node and the first node(starting point). However, the total distance is calculated traveling each node once and going back to the starting point.\
The result of total distance by using the tsp package is 2.1999999999999997 mile, which is slightly different from 2.2 mile, the result of total distance by using gurobi. This is because there can be an error when doing calculations of float values in python. Therefore, we can assume that 2.1999999999999997 mile is 2.2 mile in this case.\
You can see that the paths from two outputs are slightly different. There can be more than one possible optimal path, so the result(path) between using gurobi and tsp package can differ. Even though the paths are different, the optimal value seems to equal as 2.2 mile.